In [1]:
!which python; python -V;
from astropy import units as u
import numpy as np

from poliastro.bodies import Earth, Mars, Sun
from poliastro.twobody import Orbit

# Needed to handle TLE into Poliastro's Orbit
from tletools import TLE

# Needed for defining manouvers
from poliastro.maneuver import Maneuver
from poliastro.twobody import thrust
from poliastro.twobody.propagation import cowell
from poliastro.plotting import OrbitPlotter2D, OrbitPlotter3D

from perylune.orbit_tools import *

/home/thomson/devel/perylune/venv/bin/python
Python 3.8.5


In [2]:
# STEP 1: LAUNCH
# Let's use ANDESITE sat as an example. It was launched as shareride by RocketLab's Electron on
# the latest rideshare launch on 2020 June 13. NORAD ID is 45726. 
# TLE elements taken from n2yo website, details https://www.n2yo.com/satellite/?s=45726
tle_text = """GDASAT-1
1 45726U 20037D   20278.45278018  .00000608  00000-0  65390-4 0  9991
2 45726  97.7132  96.2906 0012962 283.6573  76.3213 14.92011802 14275"""
tle_lines = tle_text.strip().splitlines()
tle1 = TLE.from_lines(*tle_lines)
orb1 = tle1.to_orbit()

# STEP 0: DEFINE TARGET ORBIT
# Let's get the TLE orbital data for the target satellite. As an example, let's pick a dead NOAA-17 sat.
# Its TLE data can be obtained from many places, such as celestrak or n2yo (https://www.n2yo.com/satellite/?s=27453)
tle_text = """NOAA-17
1 27453U 02032A   20263.80942421 -.00000011 +00000-0 +13583-4 0  9998
2 27453 098.5909 208.3215 0011096 327.5463 032.5033 14.25072668948324"""
tle_lines = tle_text.strip().splitlines()
tle_target = TLE.from_lines(*tle_lines)
orb_target = tle_target.to_orbit()

In [3]:
# This is my attempt to do inclination change using chane_a_inc function from thrust package.

f = 0.000012 #1e-6 # km / s2

k = Earth.k.to(u.km ** 3 / u.s ** 2).value

a_0, _, inc_0, _, _, _ = orb1.classical()
a_f, _, inc_f, _, _, _ = orb_target.classical()

a_0 = a_0.value
a_f = a_f.value
inc_0 = inc_0.value
inc_f = inc_f.value

print("a, inc change: a_0=%f a_f =>%f, inc_0=%f => inc_f=%f" % (a_0, a_f, inc_0, inc_f))

# This is some serious shit.
a_d, delta_v, t_f = thrust.change_a_inc(k, a_0, a_f, inc_0, inc_f, f)
print(a_d)
print(delta_v)
print("Time of flight %f [s]" % t_f)

orb2 = orb1.propagate(t_f * u.s, method=cowell, ad=a_d, rtol=1e-6)

print("orb1=")
print_orb(orb1)
print("orb2=")
print_orb(orb2)
print("orb_target=")
print_orb(orb_target)

a, inc change: a_0=6969.800314 a_f =>7186.385343, inc_0=97.713200 => inc_f=98.590900
<function change_a_inc.<locals>.a_d at 0x7fc11e14d670>
9.547198021519023
Time of flight 795599.835127 [s]
orb1=
6961 x 6979 km x 97.7 deg (GCRS) orbit around Earth (♁) at epoch 2020-10-04T10:52:00.207552000 (UTC)
a(𝑎)=6969.80km, b=6969.79km, e=0.00, i=97.71deg raan(Ω)=96.29deg argp(𝜔)=283.66deg nu(𝜈)=76.47deg
period=5790.84s perapis=6961km(582.63km) apoapsis=6979km(600.70km)
orb2=
7172 x 7201 km x 98.6 deg (GCRS) orbit around Earth (♁) at epoch 2020-10-13T15:52:00.042678585 (UTC)
a(𝑎)=7186.34km, b=7186.33km, e=0.00, i=98.64deg raan(Ω)=85.01deg argp(𝜔)=351.80deg nu(𝜈)=-109.68deg
period=6062.80s perapis=7172km(793.82km) apoapsis=7201km(822.59km)
orb_target=
7178 x 7194 km x 98.6 deg (GCRS) orbit around Earth (♁) at epoch 2020-09-19T19:25:34.251744000 (UTC)
a(𝑎)=7186.39km, b=7186.38km, e=0.00, i=98.59deg raan(Ω)=208.32deg argp(𝜔)=327.55deg nu(𝜈)=32.57deg
period=6062.85s perapis=7178km(800.27km) apoapsis=7